In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Arrhythmia-CNN-master
%ls

/content/drive/MyDrive/Arrhythmia-CNN-master
 Arrhythmia_Classifier.ipynb    'Create Arrhythmia Dataset.ipynb'   README.md
 Arrhythmia_Classifier_O.ipynb  'Create Arrhythmia Dataset.py'      save_model/
 Arrhythmia_Classifier.py        data/


In [7]:
!pip install wfdb

     |████████████████████████████████| 133kB 7.7MB/s 


In [8]:
import numpy as np
import wfdb
import os
import pandas as pd
# import random
# random.seed(42)

In [9]:
num_labels_dict = {
    'Normal beat': 283,  # N
    'Left bundle branch block beat': 103,  # L
    'Atrial premature beat': 66,  # A
    'Atrial flutter': 20,  # (AFL (aux)
    'Atrial fibrillation': 135,  # (AFIB (aux)
    'Pre-excitation (WPW)': 21,  # (PREX (aux)
    'Premature ventricular contraction': 133,  # V
    'Ventricular bigeminy': 55,  # (B (aux)
    'Ventricular trigeminy': 13,  # (T (aux)
    'Ventricular tachycardia': 10,  # (VT (aux)
    'Idioventricular rhythm': 10,  # (IVR (aux)
    'Ventricular flutter': 10,  # (VFL (aux)
    'Fusion of ventricular and normal beat': 11,  # F
    'Second-degree heart block': 10,
    'Pacemaker rhythm': 45,  # /
    'Supraventricular tachyarrhythmia': 13,  # (SVTA (aux)
    'Right bundle branch block beat': 62,  # R
}

num_labels_we_have = {
    'N': 283,  # 'Normal beat'
    'L': 103,  # 'Left bundle branch block beat'
    'A': 66,  # 'Atrial premature beat':
    # 'V': 133, #'Premature ventricular contraction'
    # '!': 10,   #'Ventricular flutter'
    # 'F': 11, # 'Fusion of ventricular and normal beat'
    'R': 62,  # 'Right bundle branch block beat'
    'AFL': 20,
    'AFIB': 135,
    'PREX': 21,
    'B': 55,
    'T': 13,
    # '(VT'   : 10,
    'IVR': 10,
    'VFL': 10,
    'P': 45
    # '(SVTA' : 13
}


In [10]:

label_numeric_dict = {
    'N': 0,
    'A': 1,
    'AFL': 2,
    'AFIB': 3,
    'PREX': 4,
    'B': 5,
    'T': 6,
    'IVR': 7,
    'VFL': 8,
    'L': 9,
    'R': 10,
    'P': 11
}

In [11]:
def dynamic_replace(df):
    curr_aux = df['aux'].loc[0]
    for idx, x in enumerate(df['aux']):
        if x != '':
            curr_aux = x
        df.loc[idx, 'aux'] = curr_aux
    return df

def create_index_df(desired_segment_len=3600, basic_arr_path="data/mit-bih-arrhythmia-database-1.0.0"):
    arr_db = wfdb.get_record_list('mitdb')
    num_samples_in_record = 30 * 60 * 360

    # for selection and sampling
    segment_dict_ann = {}
    record_count = 0

    for _, record_id in enumerate(arr_db):
        record_path = os.path.join(basic_arr_path, str(record_id))

        ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description',
                                                                                                  'symbol',
                                                                                                  'label_store'])
        df = pd.DataFrame({'description': ann.description, 'sample': ann.sample, 'symbol': ann.symbol,
                           'label_store': ann.label_store, 'aux': ann.aux_note})
        df = dynamic_replace(df)
        counter = 0
        reset_flag = True
        allowed_labels = ['Normal beat']
        allowed_symbols = ['N']

        normal_counter = 0
        for i in range(1, df.shape[0] - 1):
            curr_label, curr_sample, curr_symbol = df.loc[i, ['description', 'sample', 'symbol']]
            if curr_label == 'Normal beat':
                normal_counter += 1
            if reset_flag:
                start_sample = curr_sample
                ann_num_start = i
                normal_counter = 0
                allowed_labels = ['Normal beat']
                allowed_symbols = ['N']
            next_label, next_sample, next_symbol = df.loc[i + 1, ['description', 'sample', 'symbol']]
            if curr_label == next_label or next_label in allowed_labels or len(allowed_labels) < 2:

                if next_label not in allowed_labels:
                    allowed_labels.append(next_label)
                    allowed_symbols.append(next_symbol)
                ann_num_end = i + 1
                counter += next_sample - curr_sample
                reset_flag = False
                if counter > desired_segment_len:
                    counter = 0
                    reset_flag = True
                    signal = wfdb.rdsamp(record_path, sampfrom=start_sample, sampto=start_sample + 3600)[0][:, 0]
                    normal_ratio = normal_counter / (ann_num_end - ann_num_start)

                    if df.loc[ann_num_start:ann_num_end]['aux'].unique().shape[0] == 1:
                        aux_seg = df.loc[ann_num_start]['aux']
                    else:
                        aux_seg = 'invalid'

                    segment_dict_ann[record_count] = [record_id, allowed_labels[-1], signal, normal_ratio,
                                                      allowed_symbols[-1], aux_seg]
                    record_count = record_count + 1
            else:
                counter = 0
                normal_counter = 0
                reset_flag = True
                allowed_labels = ['Normal beat']
                allowed_symbols = ['N']

    return segment_dict_ann

In [12]:
d_dict = create_index_df()
seg_df = pd.DataFrame.from_dict(d_dict, orient='index')
seg_df.rename(columns={0: 'record_id', 1: 'label', 2: 'signal', 3: 'normal_ratio', 4: 'symbol', 5: 'aux'}, inplace=True)

print(seg_df)
temp_=seg_df.T
type_=[]
for key_ in temp_:
  row_=temp_[key_]
  if row_[1] not in type_:
    print(row_[1])
    type_.append(row_[1])

KeyboardInterrupt: ignored

In [13]:
def preduce_(filename):
    seg_df['target'] = 'invalid'
    seg_df['aux'] = seg_df['aux'].str.lstrip('(')
    seg_df['aux'] = seg_df['aux'].str.rstrip('\x00')

    normal_aux_idx = (seg_df.aux == 'N') & (
        seg_df.symbol.apply(lambda x: True if x in num_labels_we_have.keys() else False))
    seg_df.loc[normal_aux_idx, 'target'] = seg_df.loc[normal_aux_idx, 'symbol']

    normal_label_idx = (seg_df.symbol == 'N') & (
        seg_df.aux.apply(lambda x: True if x in num_labels_we_have.keys() else False))
    seg_df.loc[normal_label_idx, 'target'] = seg_df.loc[normal_label_idx, 'aux']

    seg_df.loc[seg_df['aux'] == 'B', 'target'] = 'B'
    seg_df.loc[seg_df['aux'] == 'T', 'target'] = 'T'
    seg_df.loc[seg_df['aux'] == 'IVR', 'target'] = 'IVR'
    seg_df.loc[seg_df['aux'] == 'P', 'target'] = 'P'
    seg_df.loc[seg_df['aux'] == 'VFL', 'target'] = 'VFL'
    seg_df['target'].value_counts()


    def sample_per_sym(seg_df, cls, num):
        # filter
        target_df = seg_df.loc[seg_df[seg_df['target'] == cls].index]
        # sample
        # print(f' len df: {len(target_df)}')
        sample_df = target_df.sample(n=num)

        # add the index at the begining of the signal
        signals = np.stack(sample_df.signal.values)
        idx = 0  # TODO: tmp fix. Bar - what is this?
        return idx, sample_df, signals


    sample_df = []
    signals = []
    idx_arr = []
    for cls, num in num_labels_we_have.items():
        # print(f'target:{cls}, num: {num}')
        idx, sample_df_target, signals_target = sample_per_sym(seg_df, cls, num)
        sample_df.append(sample_df_target)
        signals.append(signals_target)
        idx_arr = np.append(idx_arr, idx)
    sample_df = pd.concat(sample_df)

    sample_df.reset_index()


    sample_df['target_str'] = sample_df['target']
    sample_df = sample_df.replace({'target': label_numeric_dict})
    sample_df.to_pickle(f"data/{filename}.pkl")

In [ ]:

# preduce_("Arrhythmia_dataset")
preduce_("Arrhythmia_dataset_yystest")
# preduce_("Arrhythmia_dataset_yystest_noseed1")
# preduce_("Arrhythmia_dataset_yystest_noseed2")
# preduce_("Arrhythmia_dataset_yystest_noseed3")
# preduce_("Arrhythmia_dataset_yystest_noseed4")
# preduce_("Arrhythmia_dataset_yystest_noseed5")

In [14]:

from torch.utils.data.dataset import Dataset

class CustomDatasetFromCSV(Dataset):
    def __init__(self, data_path, transforms_=None):
        self.df = pd.read_pickle(data_path)
        self.transforms = transforms_

    def __getitem__(self, index):
        row = self.df.iloc[index]
        signal = row['signal']
        target = row['target']
        if self.transforms is not None:
            signal = self.transforms(signal)
        signal = signal.reshape(1, signal.shape[0])
        return signal, target

    def __len__(self):
        return self.df.shape[0]

In [15]:
# dataset_O = CustomDatasetFromCSV('./data/Arrhythmia_dataset.pkl').df
# print(dataset_O.sort_index())
# print("\n\n\n================================================================================================")
dataset_0 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest.pkl').df
print(dataset_0.sort_index())
# print("\n================================================================================================")
# dataset_1 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest_noseed1.pkl').df
# print(dataset_1.sort_index())
# print("\n================================================================================================")
# dataset_2 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest_noseed2.pkl').df
# print(dataset_2.sort_index())
# print("\n================================================================================================")
# dataset_3 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest_noseed3.pkl').df
# print(dataset_3.sort_index())
# print("\n================================================================================================")
# dataset_4 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest_noseed4.pkl').df
# print(dataset_4.sort_index())
# print("\n================================================================================================")
# dataset_5 = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest_noseed5.pkl').df
# print(dataset_5.sort_index())
# print("\n================================================================================================")

     record_id                         label  ... target  target_str
0          100  Atrial premature contraction  ...      1           A
5          100                   Normal beat  ...      0           N
18         100                   Normal beat  ...      0           N
31         100                   Normal beat  ...      0           N
45         100  Atrial premature contraction  ...      1           A
...        ...                           ...  ...    ...         ...
6721       234                   Normal beat  ...      0           N
6727       234                   Normal beat  ...      0           N
6757       234                   Normal beat  ...      0           N
6758       234                   Normal beat  ...      0           N
6759       234                   Normal beat  ...      0           N

[823 rows x 8 columns]


In [17]:
temp_=dataset_0.sort_index().T

type_=[]
for key_ in temp_:
  row_=temp_[key_]
  if row_[1] not in type_:
    print(row_[1],row_[-2])
    type_.append(row_[1])

Atrial premature contraction 1
Normal beat 0
Paced beat 11
Premature ventricular contraction 5
Left bundle branch block beat 9
Right bundle branch block beat 10
Ventricular flutter wave 8
Ventricular escape beat 7
